# Benders

In [ ]:
import docplex.mp
from docplex.mp.model import Model
import cplex
from cplex.exceptions import CplexError
import math

## Sketch / Mk. 0

In [10]:
def read_instance(file_path):
    """
    Reads a CVRP instance from a file and returns the node coordinates and demands.
    """
    node_coords = {}
    demands = {}
    capacity = 0
    with open(file_path, 'r') as file:
        section = None
        for line in file:
            line = line.strip()
            if line.startswith('CAPACITY'):
                capacity = int(line.split(':')[1].strip())
            elif line.startswith('NODE_COORD_SECTION'):
                section = 'NODE_COORD_SECTION'
            elif line.startswith('DEMAND_SECTION'):
                section = 'DEMAND_SECTION'
            elif line.startswith('DEPOT_SECTION'):
                break
            elif section == 'NODE_COORD_SECTION' and line != '':
                parts = line.split()
                node_id = int(parts[0])
                x_coord = float(parts[1])
                y_coord = float(parts[2])
                node_coords[node_id] = (x_coord, y_coord)
            elif section == 'DEMAND_SECTION' and line != '':
                parts = line.split()
                node_id = int(parts[0])
                demand = float(parts[1])
                demands[node_id] = demand
    return node_coords, demands, capacity

def compute_distance_matrix(node_coords):
    """
    Computes the Euclidean distance matrix between all nodes.
    """
    nodes = list(node_coords.keys())
    distance_matrix = {}
    for i in nodes:
        distance_matrix[i] = {}
        for j in nodes:
            xi, yi = node_coords[i]
            xj, yj = node_coords[j]
            distance = math.hypot(xi - xj, yi - yj)
            distance_matrix[i][j] = distance
    return distance_matrix

In [ ]:
from docplex.mp.model import Model
import math

def read_instance(file_path):
    """
    Reads a CVRP instance from a file and returns the node coordinates, demands, and capacity.
    """
    node_coords = {}
    customers = []
    depot_coords = tuple()
    demands = {}
    capacity = 0
    with open(file_path, 'r') as file:
        section = None
        for line in file:
            line = line.strip()
            if line.startswith('CAPACITY'):
                capacity = int(line.split(':')[1].strip())
            elif line.startswith('NODE_COORD_SECTION'):
                section = 'NODE_COORD_SECTION'
            elif line.startswith('DEMAND_SECTION'):
                section = 'DEMAND_SECTION'
            elif line.startswith('DEPOT_SECTION'):
                break
            elif section == 'NODE_COORD_SECTION' and line != '':
                parts = line.split()

                node_id = int(parts[0])
                x_coord = int(parts[1])
                y_coord = int(parts[2])
                node_coords[node_id] = (x_coord, y_coord)

                if node_id == 0:
                    depot_coords = node_coords[node_id]
                else:
                    customers.append({
                        'number': node_id,
                        'x': x_coord,
                        'y': y_coord,
                        'demand': 0
                    })
            elif section == 'DEMAND_SECTION' and line != '':
                parts = line.split()

                node_id = int(parts[0])
                demand = float(parts[1])
                demands[node_id] = demand

                if node_id != 0:
                    customers[node_id]['demand'] = demand
    return node_coords, demands, capacity

def compute_distance_matrix(node_coords):
    """
    Computes the Euclidean distance matrix between all nodes.
    """
    nodes = list(node_coords.keys())
    distance_matrix = {}
    for i in nodes:
        distance_matrix[i] = {}
        xi, yi = node_coords[i]
        for j in nodes:
            xj, yj = node_coords[j]
            distance = math.hypot(xi - xj, yi - yj)
            distance_matrix[i][j] = distance
    return distance_matrix

def solve_cvrp_benders(node_coords, demands, capacity, max_vehicles):
    """
    Solves the CVRP using Benders Decomposition with flow constraints.
    """
    # Initialize variables
    nodes = list(node_coords.keys())
    customers = nodes.copy()
    customers.remove(0)  # Remove depot

    arcs = [(i, j) for i in nodes for j in nodes if i != j]

    distance_matrix = compute_distance_matrix(node_coords)
    
    # Master problem model
    master = Model("CVRP_Master")

    # Decision variables x_ij
    x_vars = {}
    for i, j in arcs:
        x_vars[i, j] = master.binary_var(name="x_{0}_{1}".format(i, j))

    # Objective function: Minimize total distance
    master.minimize(master.sum(distance_matrix[i][j] * x_vars[i, j] for i, j in arcs))

    # Constraints:

    # Each customer is visited exactly once
    for k in customers:
        master.add_constraint(master.sum(x_vars[i, k] for i in nodes if i != k) == 1, "inflow_{0}".format(k))
        master.add_constraint(master.sum(x_vars[k, j] for j in nodes if j != k) == 1, "outflow_{0}".format(k))

    # Vehicle limit constraint
    master.add_constraint(master.sum(x_vars[0, j] for j in nodes if j != 0) <= max_vehicles, "vehicle_limit")

    # Subproblem cuts will be added iteratively
    iteration = 0
    max_iterations = 100
    epsilon = 1e-6  # Tolerance for capacity violation

    while True:
        iteration += 1
        print("Iteration:", iteration)

        # Solve the master problem
        solution = master.solve(log_output=False)

        if solution is None:
            print("No feasible solution found in the master problem.")
            break

        # Extract x_ij values
        x_values = {(i, j): x_vars[i, j].solution_value for i, j in arcs}

        # Build the solution graph
        solution_arcs = [(i, j) for (i, j) in arcs if x_values[i, j] > 0.5]

        # Extract routes from the solution
        routes = extract_routes(solution_arcs, 0)

        # Check capacity feasibility
        violated_routes = []
        for route in routes:
            route_demand = sum(demands.get(node, 0) for node in route if node != 0)
            if route_demand > capacity + epsilon:
                violated_routes.append((route, route_demand))

        if not violated_routes:
            print("Optimal solution found.")
            for idx, route in enumerate(routes):
                print("Route {0}: {1}, Load: {2}".format(idx + 1, route, sum(demands.get(node, 0) for node in route)))
            break  # Feasible solution found

        # Add Benders cuts for each violated route
        for route, load in violated_routes:
            # Create a set of nodes in the route excluding the depot
            S = set(route)
            S.discard(0)

            # Create the cut: sum_{i in S, j not in S} x_ij >= 1
            cut_expr = master.sum(x_vars[i, j] for i in S for j in nodes if j not in S)
            master.add_constraint(cut_expr >= 1, "Benders_cut_{0}_{1}".format(iteration, routes.index(route)))

        # Check iteration limit
        if iteration >= max_iterations:
            print("Reached maximum number of iterations without finding a feasible solution.")
            break

def extract_routes(solution_arcs, depot):
    """
    Extracts routes from the solution arcs.
    """
    routes = []
    unvisited_arcs = solution_arcs.copy()
    while unvisited_arcs:
        route = [depot]
        current_node = depot
        while True:
            next_arc = [arc for arc in unvisited_arcs if arc[0] == current_node]
            if not next_arc:
                break
            next_node = next_arc[0][1]
            route.append(next_node)
            unvisited_arcs.remove(next_arc[0])
            current_node = next_node
            if current_node == depot:
                break
        if route[0] == depot and route[-1] == depot:
            routes.append(route)
        else:
            # Incomplete route, possible subtour
            routes.append(route)
    return routes

# Example usage
if __name__ == "__main__":
    file_path = "../Instances/small_instance_19.txt"  # Replace with your instance file path
    node_coords, demands, capacity = read_instance(file_path)
    max_vehicles = 5  # Adjust based on your problem
    solve_cvrp_benders(node_coords, demands, capacity, max_vehicles)


Iteration: 1


KeyboardInterrupt: 

# FCTP - Benders

Formulation:

1) Master Problem

\begin{equation}
\begin{aligned}
    \min \quad & z \\
    \textrm{s.t.} \quad & z \geq \sum_{s \in S}{ \sum_{d \in D}{ f_{sd} y_{sd} } } 
    + \sum_{s \in S}{(-S_s)\hat{\alpha}_s^{(k)} } 
    + \sum_{d \in D}{D_d \hat{\gamma}_d^{(k)} } 
    + \sum_{s \in S}{ \sum_{d \in D}{ (-M\hat{\omega}_{sd}^{(k)}) y_{sd} } } \\
    & \sum_{s \in S}{(-S_s)\hat{\alpha}_s^{(k)} } 
    + \sum_{d \in D}{D_d \hat{\gamma}_d^{(k)} } 
    + \sum_{s \in S}{ \sum_{d \in D}{ (-M\hat{\omega}_{sd}^{(k)}) y_{sd} } } \leq 0\\
    & y_{sd} \in \mathbb{B}, s \in S, d\in D
\end{aligned}
\end{equation}

2) Sub Problem

\begin{equation}
\begin{aligned}
    \max \quad & \sum_{s \in S}{-S_s \alpha_s} + \sum_{d \in D}{D_d\gamma_d} + \sum_{s \in S}{\sum_{d \in D}{(-M\hat{y}_{sd}) \omega_{sd} } } \\
    \textrm{s.t.} \quad & -\alpha_s + \gamma_d - \omega_{sd} \leq c_{sd}, \quad s\in S, d \in D \\
    & \alpha_s,\gamma_d,\omega_{sd} \geq 0, \quad s\in S, d \in D \\
\end{aligned}
\end{equation}

In [1]:
import sys
import numpy as np
import cplex
import matplotlib.pyplot as plt
from cplex.exceptions import CplexError

### Read Data & Aux Classes

In [2]:
# class ProblemData:
#     nS = 4  # Number of sources
#     nD = 3  # Number of destinations
#     seed = 39
#     BigM = None
#     S = None  # Supplies
#     D = None  # Demands
#     C = None  # Costs
#     f = None  # Fixed charges

#     @staticmethod
#     def populate_parameters():
#         np.random.seed(ProblemData.seed)
#         nS = ProblemData.nS
#         nD = ProblemData.nD

#         # Generate demands
#         D = np.random.randint(20, 150, size=nD)
#         Dsum = np.sum(D)

#         # Generate supplies approximately equal to total demand
#         S = np.random.randint(150, 190, size=nS)
#         Ssum = np.sum(S)
#         S = np.round((S * Dsum) / Ssum).astype(int) + 1

#         # Calculate Big M
#         BigM = max(np.max(D), np.max(S))

#         # Generate cost matrix C[s][d]
#         C = np.random.randint(1, 10, size=(nS, nD))

#         # Generate fixed charge matrix f[s][d]
#         f = np.random.randint(15, 30, size=(nS, nD))

#         # Assign to class variables
#         ProblemData.S = S
#         ProblemData.D = D
#         ProblemData.C = C
#         ProblemData.f = f
#         ProblemData.BigM = BigM


In [3]:
def plot_bounds(iterations, UB_history, LB_history, MP_Obj_history, SP_Obj_history):
    #%matplotlib inline
    
    plt.figure(figsize=(16, 8))
    iterations = range(1, iterations+1)

    plt.plot(iterations, UB_history[1:], label='Upper Bound (UB)')
    plt.plot(iterations, LB_history[1:], label='Lower Bound (LB)')
    plt.plot(iterations, MP_Obj_history[1:], label='Master Problem Objective (MP_Obj)')
    # plt.plot(iterations, SP_Obj_history, label='Subproblem Objective (SP_Obj)', marker='x')
    plt.xlabel('Iteration')
    plt.ylabel('Objective Value')
    plt.title('Evolution of Bounds and Objectives over Iterations')
    plt.legend()
    plt.grid(True)
    plt.show()

In [4]:
class ProblemData:
    def __init__(self, nS=4, nD=3, seed=39) -> None:
        self.nS = nS  # Number of sources
        self.nD = nD  # Number of destinations
        self.seed = seed # Random seed
        self.BigM = None
        self.S = None  # Supplies
        self.D = None  # Demands
        self.C = None  # Costs
        self.f = None  # Fixed charges

    def populate_parameters(self):
        np.random.seed(self.seed)

        # Generate demands
        self.D = np.random.randint(20, 150, size=self.nD).tolist()
        Dsum = np.sum(self.D)

        # Generate supplies approximately equal to total demand
        self.S = np.random.randint(150, 190, size=self.nS)
        Ssum = np.sum(self.S)
        self.S = np.round((self.S * Dsum) / Ssum).astype(int) + 1
        self.S = self.S.tolist()

        # Calculate Big M
        self.BigM = int(max(np.max(self.D), np.max(self.S)))

        # Generate cost matrix C[s][d]
        self.C = np.random.randint(1, 10, size=(self.nS, self.nD)).tolist()

        # Generate fixed charge matrix f[s][d]
        self.f = np.random.randint(15, 30, size=(self.nS, self.nD)).tolist()


In [5]:
class Cut:
    def __init__(self, data):
        self.add_optimality_cut = False
        self.add_feasibility_cut = False
        nS = data.nS
        nD = data.nD
        self.Us = np.zeros(nS).tolist()
        self.Vs = np.zeros(nD).tolist()
        self.Ws = np.zeros((nS, nD)).tolist()
        self.Us_ray = np.zeros(nS).tolist()
        self.Vs_ray = np.zeros(nD).tolist()
        self.Ws_ray = np.zeros((nS, nD)).tolist()
    
    def debug(self):
        if self.add_optimality_cut:
            print(f'Feasibility-Cut: {self.add_optimality_cut, self.add_feasibility_cut}')
            print(f'Alpha: {self.Us}')
            print(f'Gamma: {self.Vs}')
            print(f'Omega: {self.Ws}')
        elif self.add_feasibility_cut:
            print(f'Optimality-Cut: {self.add_optimality_cut, self.add_feasibility_cut}')
            print(f'Alpha_ray: {self.Us_ray}')
            print(f'Gamma_ray: {self.Vs_ray}')
            print(f'Omega_ray: {self.Ws_ray}')
    
    def update_rays(self, Us_ray, Vs_ray, Ws_ray):
        if self.add_feasibility_cut:
            self.Us_ray, self.Vs_ray, self.Ws_ray = Us_ray, Vs_ray, Ws_ray

    def update(self, Us, Vs, Ws):
        if self.add_optimality_cut:
            self.Us, self.Vs, self.Ws = Us, Vs, Ws


### Master Problem

In [6]:
def MasterProblem(Ys, Cuts, data):
    nS = data.nS
    nD = data.nD
    S = data.S
    D = data.D
    C = data.C
    f = data.f
    BigM = data.BigM

    try:
        # Initialize CPLEX problem
        master = cplex.Cplex()
        master.objective.set_sense(master.objective.sense.minimize)
        master.set_log_stream(None)
        master.set_error_stream(None)
        master.set_warning_stream(None)
        master.set_results_stream(None)

        # Decision variables Y_sd
        Y_names = ["Y_%d_%d" % (s, d) for s in range(nS) for d in range(nD)]
        Y_obj = [0] * (nS * nD)
        Y_lb = [0] * (nS * nD)
        Y_ub = [1] * (nS * nD)
        Y_types = ["B"] * (nS * nD)
        master.variables.add(obj=Y_obj, lb=Y_lb, ub=Y_ub, types=Y_types, names=Y_names)

        # Decision variable Z
        master.variables.add(obj=[1.0], lb=[-cplex.infinity], ub=[cplex.infinity], names=["Z"])

        # Objective function: Minimize Z
        master.objective.set_linear("Z", 1.0)

        # Add cuts
        for cut in Cuts:
            if cut.add_optimality_cut:
                # Optimality cut: Z >= ...
                expr = [["Z"], [1.0]]
                for s in range(nS):
                    for d in range(nD):
                        expr[0].append(f"Y_{s}_{d}")
                        # Move to LHS by multiplying by -1
                        expr[1].extend([-f[s][d] + BigM * cut.Ws[s][d]])
                
                rhs = sum(-S[s] * cut.Us[s] for s in range(nS)) + sum(D[d] * cut.Vs[d] for d in range(nD)) + sum(expr[1])
                master.linear_constraints.add(lin_expr=[expr], senses=["G"], rhs=[rhs])

            if cut.add_feasibility_cut:
                # Feasibility cut: expr <= 0
                rhs = 0.0
                expr = [[], []]
                for s in range(nS):
                    for d in range(nD):
                        expr[0].extend([f"Y_{s}_{d}"])
                        expr[1].extend([-BigM * cut.Ws_ray[s][d]])
                rhs -= sum(-S[s] * cut.Us_ray[s] for s in range(nS)) + sum(D[d] * cut.Vs_ray[d] for d in range(nD))
                master.linear_constraints.add(lin_expr=[expr], senses=["L"], rhs=[rhs])
        master.write("master.lp")
        # Solve the master problem
        master.solve()
        obj = master.solution.get_objective_value()
        print("\t MP obj value is:", obj)

        # Update Ys
        var_names = master.variables.get_names()
        for s in range(nS):
            for d in range(nD):
                idx = var_names.index("Y_%d_%d" % (s, d))
                Ys[s][d] = master.solution.get_values(idx)

        return obj

    except CplexError as e:
        print("Cplex Error:", e)
        return None


### Sub Problem

In [7]:
def SubProblem(Ys, Cuts, data):
    nS = data.nS
    nD = data.nD
    S = data.S
    D = data.D
    C = data.C
    f = data.f
    BigM = data.BigM

    # print(f"nS:{nS}, nD:{nD},")
    try:
        # Initialize CPLEX problem
        subprob = cplex.Cplex()
        subprob.objective.set_sense(subprob.objective.sense.maximize)
        subprob.set_log_stream(None)
        subprob.set_error_stream(None)
        subprob.set_warning_stream(None)
        subprob.set_results_stream(None)

        # Decision variables alpha_s
        U_names = [f"U_{s}" for s in range(nS)]
        subprob.variables.add(obj=[-S[s] for s in range(nS)],
                              lb=[0]*nS, ub=[cplex.infinity]*nS, names=U_names)

        # Decision variables gamma_d
        V_names = [f"V_{d}" for d in range(nD)]
        subprob.variables.add(obj=[D[d] for d in range(nD)],
                              lb=[0]*nD, ub=[cplex.infinity]*nD, names=V_names)

        # Decision variables omega_sd
        W_names = []
        W_obj = []
        W_lb = []
        W_ub = []
        for s in range(nS):
            for d in range(nD):
                W_names.append(f"W_{s}_{d}")
                W_obj.append(-BigM * Ys[s][d] ) # + f[s][d] * Ys[s][d]
                W_lb.append(0)
                W_ub.append(cplex.infinity)
        subprob.variables.add(obj=W_obj, lb=W_lb, ub=W_ub, names=W_names)

        # Constraints: -U_s + V_d - W_sd <= C_sd
        constraints = []
        senses = []
        rhs = []
        constraint_names = []
        for s in range(nS):
            for d in range(nD):
                row = [[f"U_{s}", f"V_{d}", f"W_{s}_{d}"],
                       [-1.0, 1.0, -1.0]]
                constraints.append(row)
                senses.append("L")
                rhs.append(C[s][d])
                constraint_names.append(f"c_{s}_{d}")
        subprob.linear_constraints.add(lin_expr=constraints, senses=senses,
                                       rhs=rhs, names=constraint_names)

        # Disable presolve to detect unboundedness
        subprob.parameters.preprocessing.presolve.set(0)

        # subprob.write("subprob.lp")
        # sys.exit(1)
        # Solve the subproblem
        subprob.solve()

        status = subprob.solution.get_status()
        print("\t The Subproblem is:", subprob.solution.get_status_string(status))

        new_cut = Cut(data=data)
        var_names = subprob.variables.get_names()
        SP_feasible = None

        if status == subprob.solution.status.unbounded:
            # Subproblem is unbounded, generate feasibility cut
            new_cut.add_feasibility_cut = True
            ray = subprob.solution.advanced.get_ray()
            # print(f'Ray:{ray}\n\n')
            # Extract values for U variables
            for s in range(nS):
                idx = var_names.index(f"U_{s}")
                new_cut.Us_ray[s] = ray[idx]

            # Extract values for V variables
            for d in range(nD):
                idx = var_names.index(f"V_{d}")
                new_cut.Vs_ray[d] = ray[idx]

            # Extract values for W variables
            for s in range(nS):
                for d in range(nD):
                    idx = var_names.index(f"W_{s}_{d}")
                    new_cut.Ws_ray[s][d] = ray[idx]

            # new_cut.update_rays(new_cut.Us_ray, new_cut.Vs_ray, new_cut.Ws_ray)   
            # new_cut.debug()
            Cuts.append(new_cut)
            SP_feasible = False
            
            return cplex.infinity, SP_feasible  # Indicate unboundedness

        else:
            # Subproblem is feasible, generate optimality cut
            new_cut.add_optimality_cut = True
            obj = subprob.solution.get_objective_value()
            print("\t SP obj value is:", obj)

            # Extract values for U variables
            for s in range(nS):
                idx = var_names.index(f"U_{s}")
                new_cut.Us[s] = subprob.solution.get_values(idx)

            # Extract values for V variables
            for d in range(nD):
                idx = var_names.index(f"V_{d}")
                new_cut.Vs[d] = subprob.solution.get_values(idx)

            # Extract values for W variables
            for s in range(nS):
                for d in range(nD):
                    idx = var_names.index(f"W_{s}_{d}")
                    new_cut.Ws[s][d] = subprob.solution.get_values(idx)
            # new_cut.debug()
            Cuts.append(new_cut)
            SP_feasible = True
            
            return obj, SP_feasible

    except CplexError as e:
        print("Cplex Error:", e)
        return None


### Main Benders Algorithm

In [8]:
def compute_total_cost(Y, data):
    nS = data.nS
    nD = data.nD
    S = data.S
    D = data.D
    f = data.f
    c = data.C

    total_cost = 0.0
    for s in range(nS):
        for d in range(nD):
            if Y[s][d] > 0.5:  # Assuming Ys[s][d] is binary
                total_cost += f[s][d]  # Fixed charge
                # Since we have selected y_sd, we need to compute flow x_sd
                # For the fixed charge transportation problem, flow is determined by supply and demand
                # For simplicity, assume maximum flow between s and d
                flow = min(S[s], D[d])
                total_cost += c[s][d] * flow
    return total_cost

In [ ]:
def main():
    # Set problem parameters
    PD = ProblemData(nS=10, nD=6, seed=5)
    # if len(sys.argv) >= 4:
    #     ProblemData.nS = int(sys.argv[1])
    #     ProblemData.nD = int(sys.argv[2])
    #     ProblemData.seed = int(sys.argv[3])
    # else:
    #     ProblemData.nS = 4
    #     ProblemData.nD = 3
    #     ProblemData.seed = 39

    # Populate problem data
    PD.populate_parameters()

    nS = PD.nS
    nD = PD.nD
    S = PD.S # Supply at each source
    D = PD.D # Demand at each destination
    C = PD.C # Transportation Costs
    f = PD.f # Fixed Charges
    BigM = PD.BigM

    # Initialization: Create y variables (binary)
    Y = [[1]*nD for _ in range(nS)] # np.ones((nS, nD)) Initialize Ys to 1

    UB = 1e7
    LB = -1e7

    # Main loop for benders decomposition
    cuts = []
    iter_count = 0

    ub_hist = [UB]
    lb_hist = [LB]
    mp_hist, sp_hist = [LB], [UB]

    while abs(UB - LB) > 1e-3:
        iter_count += 1
        print("\n\n \t\t Iteration:", iter_count, "\t UB:", UB, "\t LB:", LB)
        # for c in cuts: c.debug()
        
        # Solve subproblem taking current y values
        SP_obj, SP_feasible = SubProblem(Y, cuts, PD)
        if SP_obj is None:
            print("Subproblem failed.")
            break
        sp_hist.append(SP_obj)


        # Solve master problem with new cuts
        MP_obj = MasterProblem(Y, cuts, PD)
        if MP_obj is None:
            print("Master problem failed.")
            break
        mp_hist.append(MP_obj)

        # Update bounds
        # UB = min(UB, SP_obj)
        if SP_feasible:
            # Compute the total cost using Ys and SP solution
            total_cost = compute_total_cost(Y, PD)
            # Update UB if total_cost is better
            UB = min(UB, total_cost)
        LB = MP_obj

        ub_hist.append(UB)
        lb_hist.append(LB)

        if iter_count == 1000: break

    print("Optimal value:", UB)
    print("Optimal Ys:", Y)
    plot_bounds(iter_count, ub_hist, lb_hist, mp_hist, sp_hist)
if __name__ == "__main__":
    main()




 		 Iteration: 1 	 UB: 10000000.0 	 LB: -10000000.0
	 The Subproblem is: optimal
	 SP obj value is: 1044.0
	 MP obj value is: -259.0


 		 Iteration: 2 	 UB: 0.0 	 LB: -259.0
	 The Subproblem is: unbounded
	 MP obj value is: -199.0


 		 Iteration: 3 	 UB: 0.0 	 LB: -199.0
	 The Subproblem is: unbounded
	 MP obj value is: -195.0


 		 Iteration: 4 	 UB: 0.0 	 LB: -195.0
	 The Subproblem is: unbounded
	 MP obj value is: -194.0


 		 Iteration: 5 	 UB: 0.0 	 LB: -194.0
	 The Subproblem is: unbounded
	 MP obj value is: -194.0


 		 Iteration: 6 	 UB: 0.0 	 LB: -194.0
	 The Subproblem is: unbounded
	 MP obj value is: -188.0


 		 Iteration: 7 	 UB: 0.0 	 LB: -188.0
	 The Subproblem is: unbounded
	 MP obj value is: -187.0


 		 Iteration: 8 	 UB: 0.0 	 LB: -187.0
	 The Subproblem is: unbounded
	 MP obj value is: -184.0


 		 Iteration: 9 	 UB: 0.0 	 LB: -184.0
	 The Subproblem is: unbounded
	 MP obj value is: -177.0


 		 Iteration: 10 	 UB: 0.0 	 LB: -177.0
	 The Subproblem is: unbounded

# Benders based on ATSP example

### Read data

In [ ]:
class ProblemData:
    def __init__(self, nS=4, nD=3, seed=39) -> None:
        self.nS = nS  # Number of sources
        self.nD = nD  # Number of destinations
        self.seed = seed # Random seed
        self.BigM = None
        self.S = None  # Supplies
        self.D = None  # Demands
        self.C = None  # Costs
        self.f = None  # Fixed charges

    def populate_parameters(self):
        np.random.seed(self.seed)

        # Generate demands
        self.D = np.random.randint(20, 150, size=self.nD).tolist()
        Dsum = np.sum(self.D)

        # Generate supplies approximately equal to total demand
        self.S = np.random.randint(150, 190, size=self.nS)
        Ssum = np.sum(self.S)
        self.S = np.round((self.S * Dsum) / Ssum).astype(int) + 1
        self.S = self.S.tolist()

        # Calculate Big M
        self.BigM = int(max(np.max(self.D), np.max(self.S)))

        # Generate cost matrix C[s][d]
        self.C = np.random.randint(1, 10, size=(self.nS, self.nD)).tolist()

        # Generate fixed charge matrix f[s][d]
        self.f = np.random.randint(15, 30, size=(self.nS, self.nD)).tolist()


### Master Problem

In [ ]:
def create_master_ilp(cpx, y, data):
    """Creates the master ILP for the Fixed Cost Transportation Problem (FCTP)."""

    fixed_cost = data.fixed_cost    # f_sd
    num_suppliers = data.num_suppliers
    num_demands = data.num_demands

    cpx.objective.set_sense(cpx.objective.sense.minimize)

    # Create y(sd) variables for each (s, d) in S x D
    for s in range(num_suppliers):
        y.append([])
        for d in range(num_demands):
            var_name = f"y_{s}.{d}"
            y[s].append(cpx.variables.get_num())
            cpx.variables.add(obj=[fixed_cost[s][d]],
                              lb=[0.0], ub=[1.0], types=["B"],
                              names=[var_name])

    # No need to create X(sd) variables in the master problem since they are handled in the subproblem

    # The master problem only includes y(sd) variables and Benders' cuts will be added via callbacks


### Worker Class (subproblem)

In [ ]:
class WorkerLP():
    """This class builds the worker LP for the Fixed Cost Transportation Problem (FCTP) and allows to separate violated Benders' cuts."""

    def __init__(self, num_suppliers, num_demands, data):
        # Initialize CPLEX instance for the worker LP
        cpx = cplex.Cplex()
        cpx.set_results_stream(None)
        cpx.set_log_stream(None)

        # Set to maximize as per subproblem formulation
        cpx.objective.set_sense(cpx.objective.sense.maximize)

        # Create dual variables:
        # alpha_s for capacity constraints (s ∈ S)
        # gamma_d for demand constraints (d ∈ D)
        # omega_sd for linking constraints (s ∈ S, d ∈ D)
        self.alpha = []
        self.gamma = []
        self.omega = []

        # Create alpha_s variables
        for s in range(num_suppliers):
            var_name = f"alpha_{s}"
            self.alpha.append(cpx.variables.get_num())
            cpx.variables.add(obj=[-1 * data.S[s]],  # Coefficient in objective
                              lb=[0.0],
                              ub=[cplex.infinity],
                              names=[var_name])

        # Create gamma_d variables
        for d in range(num_demands):
            var_name = f"gamma_{d}"
            self.gamma.append(cpx.variables.get_num())
            cpx.variables.add(obj=[data.D[d]],  # Coefficient in objective
                              lb=[0.0],
                              ub=[cplex.infinity],
                              names=[var_name])

        # Create omega_sd variables
        for s in range(num_suppliers):
            self.omega.append([])
            for d in range(num_demands):
                var_name = f"omega_{s}.{d}"
                self.omega[s].append(cpx.variables.get_num())
                cpx.variables.add(obj=[-data.BigM],  # Coefficient in objective 
                                  #TODO obj=[-M[s][d]] change BigM to matrix for stronger bounds
                                  lb=[0.0],
                                  ub=[cplex.infinity],
                                  names=[var_name])

        # Add constraints: -alpha_s + gamma_d - omega_sd <= c_sd, for all s, d
        for s in range(num_suppliers):
            for d in range(num_demands):
                thevars = []
                thecoefs = []
                thevars.append(self.alpha[s])
                thecoefs.append(-1.0)
                thevars.append(self.gamma[d])
                thecoefs.append(1.0)
                thevars.append(self.omega[s][d])
                thecoefs.append(-1.0)
                cpx.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(thevars, thecoefs)],
                    senses=["L"],
                    rhs=[data.C[s][d]])  # c_sd

        self.cpx = cpx
        self.num_suppliers = num_suppliers
        self.num_demands = num_demands
        self.data = data
        self.cut_lhs = None
        self.cut_rhs = None

    def separate(self, y_sol, y):
        """Separates Benders' cuts violated by the current y solution."""

        cpx = self.cpx
        num_suppliers = self.num_suppliers
        num_demands = self.num_demands
        violated_cut_found = False

        # Update the objective function:
        # max (-sum_s S_s * alpha_s + sum_d D_d * gamma_d - sum_{s,d} M_sd * y_sd * omega_sd )
        thevars = []
        thecoefs = []
        for s in range(num_suppliers):
            thevars.append(self.alpha[s])
            thecoefs.append(-1 * self.data.S[s])
        for d in range(num_demands):
            thevars.append(self.gamma[d])
            thecoefs.append(self.data.D[d])
        for s in range(num_suppliers):
            for d in range(num_demands):
                thevars.append(self.omega[s][d])
                thecoefs.append(-self.data.BigM * y_sol[s][d])

        cpx.objective.set_linear(zip(thevars, thecoefs))

        # Solve the worker LP
        cpx.solve()

        # Check for optimality to determine if a cut is violated
        status = cpx.solution.get_status()
        if status == cpx.solution.status.optimal:
            obj_val = cpx.solution.get_objective_value()
            if obj_val > 1e-06:
                # A violated Benders' cut is found
                # Retrieve the dual variables
                duals = cpx.solution.get_dual_values()

                # Construct the Benders' cut based on dual variables
                # The master problem constraint should be: 
                # z >= sum_s sum_d f_sd y_sd + sum_s (-S_s) * alpha_s + sum_d D_d * gamma_d + sum_s sum_d (-M_sd * omega_sd) y_sd
                
                #FIXME #TODO : Remove sum_s sum_d f_sd y_sd and add to obj?
                # z >= sum_s (-S_s) * alpha_s + sum_d D_d * gamma_d + sum_s sum_d (-M_sd * omega_sd) y_sd 

                # Extract dual variables for capacity constraints (alpha_s)
                alpha_duals = duals[:num_suppliers]

                # Extract dual variables for demand constraints (gamma_d)
                gamma_duals = duals[num_suppliers:num_suppliers + num_demands]

                # Extract dual variables for linking constraints (omega_sd)
                omega_duals = duals[num_suppliers + num_demands:]

                # Construct the cut coefficients for y_sd
                cut_coeffs = []
                cut_vars = []
                for s in range(num_suppliers):
                    for d in range(num_demands):
                        coef = -self.data.BigM * omega_duals[s * num_demands + d]
                        if abs(coef) > 1e-06:
                            cut_coeffs.append(coef)
                            cut_vars.append(y[s][d])

                # Construct the right-hand side of the cut
                # z >= sum_s sum_d f_sd y_sd + sum_s (-S_s * alpha_s) + sum_d (D_d * gamma_d) + sum_s sum_d (-M_sd * omega_sd) y_sd
                # Rearranged as:
                # z - sum_s sum_d f_sd y_sd - sum_s (-S_s * alpha_s) - sum_d (D_d * gamma_d) - sum_s sum_d (-M_sd * omega_sd) y_sd >= 0
                # Therefore, the cut can be represented as:
                # z - sum_s sum_d f_sd y_sd - sum_s (-S_s * alpha_s) - sum_d (D_d * gamma_d) >= sum_s sum_d (-M_sd * omega_sd) y_sd

                # However, to implement this, it's simpler to express it in terms of the current master variables and set z accordingly.

                # For simplicity, we define a linear combination involving y_sd variables
                # and set the constraint such that:
                # z >= (fixed costs) + (dual variables terms)

                # Assuming 'z' is the auxiliary variable in the master problem
                # and it's already part of the master problem variables.

                # Therefore, add the cut:
                # z >= sum_s sum_d f_sd y_sd + sum_s (-S_s * alpha_s) + sum_d (D_d * gamma_d) + sum_s sum_d (-M_sd * omega_sd) y_sd

                # Translate this into a constraint:
                # z - sum_s sum_d (f_sd + M_sd * omega_sd) y_sd >= sum_s (-S_s * alpha_s) + sum_d (D_d * gamma_d)

                # Construct the left-hand side coefficients
                cut_vars_full = []
                cut_coefs_full = []

                # Coefficients for y_sd
                for s in range(num_suppliers):
                    for d in range(num_demands):
                        coef = -(self.data.f[s][d] + self.data.BigM * omega_duals[s * num_demands + d])
                        if abs(coef) > 1e-06:
                            cut_vars_full.append(y[s][d])
                            cut_coefs_full.append(coef)

                # Coefficient for z (to be added externally; assume 'z' is indexed appropriately)
                # Here, we need to reference 'z' in the master problem.
                # For simplicity, assume 'z' is the first variable in the master problem.
                # Adjust the index accordingly based on the actual master problem variable indexing.
                # Example:
                # z_index = 0
                # cut_vars_full.append(z)
                # cut_coefs_full.append(1.0)

                # Since 'z' is not part of y_sd, additional handling is required.
                # One approach is to add 'z' as a variable in the master problem and reference it here.
                # Ensure that 'z' is defined and accessible.

                # Placeholder: Assuming 'z' is accessible as 'z_var'
                # In practice, pass 'z' to the WorkerLP or manage its indexing accordingly.

                # For demonstration, let's assume 'z' is the first variable
                # and has index 'z_idx' in the master problem
                # You need to adjust 'z_idx' based on your master problem implementation
                z_idx = 0  # Placeholder index for 'z'

                # Add 'z' to the cut
                # cut_vars_full.append(z_var)
                # cut_coefs_full.append(1.0)

                # Create the cut as a SparsePair
                # Since 'z' is not handled here, ensure it is integrated properly in the master problem
                # This requires coordination between the worker and master problem implementations

                # For simplicity, we assume that 'z' is handled externally and focus on the y_sd coefficients

                # Add the cut to the master problem
                # self.cut_lhs = cplex.SparsePair(ind=cut_vars_full, val=cut_coefs_full)
                # self.cut_rhs = sum([-data.capacities[s] * alpha_duals[s] for s in range(num_suppliers)]) + \
                #                sum([data.demands[d] * gamma_duals[d] for d in range(num_demands)])

                # Create the SparsePair for the y_sd variables
                cut_lhs = cpx.SparsePair(ind=cut_vars_full, val=cut_coefs_full)

                # Compute the right-hand side of the cut
                cut_rhs = 0.0
                for s in range(num_suppliers):
                    cut_rhs += -self.data.S[s] * alpha_duals[s]
                for d in range(num_demands):
                    cut_rhs += self.data.D[d] * gamma_duals[d]

                self.cut_lhs = cut_lhs
                self.cut_rhs = cut_rhs
                violated_cut_found = True

        return violated_cut_found


### Callback Class

In [ ]:
class FCTPCallback():
    """Callback function for the Fixed Cost Transportation Problem (FCTP).

    This callback can do two different things:
       - Separate Benders' cuts at fractional solutions as user cuts
       - Separate Benders' cuts at integer solutions as lazy constraints

    Everything is setup in the invoke function that
    is called by CPLEX.
    """

    def __init__(self, num_threads, num_suppliers, num_demands, y, z, data):
        self.num_threads = num_threads
        self.cutlhs = None
        self.cutrhs = None
        self.num_suppliers = num_suppliers
        self.num_demands = num_demands
        self.y = y
        self.z = z
        self.data = data
        # Create workerLP for Benders' cuts separation
        self.workers = [None] * num_threads

    def separate_user_cuts(self, context, worker):
        """Separate Benders' cuts at fractional solutions as user cuts."""
        # Get the current y solution (fractional)
        y_sol = []
        for s in range(self.num_suppliers):
            for d in range(self.num_demands):
                y_val = context.get_relaxation_point(self.y[s][d])
                y_sol.append(y_val)
        y_sol_matrix = [y_sol[i * self.num_demands:(i + 1) * self.num_demands] for i in range(self.num_suppliers)]

        # Reshape y_sol into a matrix [s][d]
        y_sol_reshaped = y_sol_matrix

        # Benders' cut separation
        if worker.separate(y_sol_reshaped, self.y):
            cutlhs = worker.cut_lhs
            cutrhs = worker.cut_rhs
            # Assuming 'z' is the first variable in the master problem
            # Adjust 'z_idx' based on actual master problem variable indexing
            z_idx = 0
            # Add 'z' variable to the cut
            cutlhs.ind.append(self.z)
            cutlhs.val.append(1.0)
            # Add the existing y_sd coefficients
            # This forms: z - sum(s,d) (f_sd + M_sd * omega_sd) y_sd >= RHS
            context.add_user_cut(cut=cutlhs, sense='G', rhs=cutrhs,
                                 cutmanagement=cplex.callbacks.UserCutCallback.use_cut.purge,
                                 local=False)

    def separate_lazy_constraints(self, context, worker):
        """Separate Benders' cuts at integer solutions as lazy constraints."""
        # We only work with bounded models
        if not context.is_candidate_point():
            raise Exception('Unbounded solution')

        # Get the current y solution (integer)
        y_sol = []
        for s in range(self.num_suppliers):
            for d in range(self.num_demands):
                y_val = context.get_candidate_point(self.y[s][d])
                y_sol.append(y_val)
        y_sol_matrix = [y_sol[i * self.num_demands:(i + 1) * self.num_demands] for i in range(self.num_suppliers)]

        # Reshape y_sol into a matrix [s][d]
        y_sol_reshaped = y_sol_matrix

        # Benders' cut separation
        if worker.separate(y_sol_reshaped, self.y):
            cutlhs = worker.cut_lhs
            cutrhs = worker.cut_rhs
            # Assuming 'z' is the first variable in the master problem
            # Adjust 'z_idx' based on actual master problem variable indexing
            z_idx = 0
            # Add 'z' variable to the cut
            cutlhs.ind.append(self.z)
            cutlhs.val.append(1.0)
            # Add the existing y_sd coefficients
            # This forms: z - sum(s,d) (f_sd + M_sd * omega_sd) y_sd >= RHS
            context.add_lazy_constraint(cut=cutlhs, sense='G', rhs=cutrhs)

    def invoke(self, context):
        """Callback method called by CPLEX."""
        try:
            thread_id = context.get_int_info(
                cplex.callbacks.Context.info.thread_id)
            if context.get_id() == cplex.callbacks.Context.id.thread_up:
                self.workers[thread_id] = WorkerLP(self.num_suppliers, self.num_demands, self.data.BigM)
            elif context.get_id() == cplex.callbacks.Context.id.thread_down:
                self.workers[thread_id] = None
            elif context.get_id() == cplex.callbacks.Context.id.relaxation:
                self.separate_user_cuts(context, self.workers[thread_id])
            elif context.get_id() == cplex.callbacks.Context.id.candidate:
                self.separate_lazy_constraints(context, self.workers[thread_id])
            else:
                print("Callback called in an unexpected context {}".format(
                    context.get_id()))
        except Exception as e:
            print(f"#### Exception in callback: {e}")
            traceback.print_exc()
            raise


### Benders orchestrator

In [ ]:
def bendersfctp(sep_frac_sols, data):
    """Solves a Fixed Cost Transportation Problem (FCTP) instance through Benders decomposition."""
    print("Benders' cuts separated to cut off: ", end=' ')
    if sep_frac_sols == "1":
        print("Integer and fractional infeasible solutions.")
        sep_frac_sols = True
    elif sep_frac_sols == "0":
        print("Only integer infeasible solutions.")
        sep_frac_sols = False
    else:
        raise ValueError('sep_frac_sols must be one of "0" or "1"')


    # Create master ILP
    cpx = cplex.Cplex()
    y = []
    create_master_ilp(cpx, y, data)
    num_suppliers = data.nS
    num_demands = data.nD

    # Add auxiliary variable z to the master problem
    z_var_name = "z"
    cpx.variables.add(obj=[1.0],
                      lb=[-cplex.infinity],
                      ub=[cplex.infinity],
                      types=["C"],
                      names=[z_var_name])
    z_index = cpx.variables.get_num() - 1  # Assuming z is the last variable

    # Update the master problem's objective to minimize z
    cpx.objective.set_linear([(z_index, 1.0)])
    cpx.objective.set_sense(cpx.objective.sense.minimize)

    # Set number of threads
    num_threads = cpx.get_num_cores() // 2
    cpx.parameters.threads.set(num_threads)

    # Initialize and register callback
    # Pass the z variable index to the callback
    fctp_cb = FCTPCallback(num_threads, num_suppliers, num_demands, y, z_index, data)
    contextmask = cplex.callbacks.Context.id.thread_up
    contextmask |= cplex.callbacks.Context.id.thread_down
    contextmask |= cplex.callbacks.Context.id.candidate
    if sep_frac_sols:
        contextmask |= cplex.callbacks.Context.id.relaxation
    cpx.set_callback(fctp_cb, contextmask)

    # Solve the master ILP
    cpx.solve()

    # Retrieve and display the solution
    solution = cpx.solution
    print()
    print("Solution status: ", solution.get_status())
    print("Objective value: ", solution.get_objective_value())

    if solution.get_status() == solution.status.MIP_optimal:
        # Extract the optimal facility openings and shipments
        opened_facilities = []
        for s in range(num_suppliers):
            for d in range(num_demands):
                y_val = solution.get_values(y[s][d])
                if y_val > 0.5:
                    opened_facilities.append((s, d))
        print("Opened Facilities (s, d):", opened_facilities)

        # Extract the z value
        z_val = solution.get_values(z_index)
        print("Objective value (z):", z_val)

        # Shipment allocation can be derived from the subproblem or additional data
        # For simplicity, assuming that shipment allocations are managed externally
    else:
        print("Solution status is not optimal")


### Run

In [ ]:
def main():
    """Set default arguments and parse command line."""
    # if len(sys.argv) != 2 and len(sys.argv) != 3:
    #     usage()
    #     sys.exit(-1)
    # if sys.argv[1] not in ["0", "1"]:
    #     usage()
    #     sys.exit(-1)
    # if len(sys.argv) == 3:
    #     filename = sys.argv[2]
    # else:
    #   filename = "path_to_default_fctp_data.dat"  # Update to appropriate default

    # Read problem data from data file
    data = ProblemData()
    data.populate_parameters()
    bendersfctp(sep_frac_sols="1", data=data)
